In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import zipfile
# with zipfile.ZipFile("heart-diseaseuci.zip", "r") as f:
#     f.extractall("heart.csv")

In [6]:
df = pd.read_csv("E:/Codes/Machine_Learning/heart_disease_data/heart.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [8]:
df.shape

(303, 14)

In [14]:
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
print(X_train.shape)
print(X_test.shape)

(242, 13)
(61, 13)


In [26]:
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
svc = SVC()
lr = LogisticRegression()

In [28]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("Accuracy Score Randonm Forest: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score Randonm Forest: 0.8688524590163934


In [30]:
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
print("Accuracy Score Gradient Boosting: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score Gradient Boosting: 0.7704918032786885


In [34]:
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print("Accuracy Score SVC: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score SVC: 0.7049180327868853


In [36]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("Accuracy Score Logistic Regression: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score Logistic Regression: 0.8852459016393442


C:\Users\atifs\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Hyperparameter Tunning (Random Forest)

In [39]:
rf = RandomForestClassifier(max_samples=0.75, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("Accuracy Score Random Forest(Tunned): {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score Random Forest(Tunned): 0.9016393442622951


In [41]:
from sklearn.model_selection import cross_val_score

In [47]:
np.mean(cross_val_score(RandomForestClassifier(max_samples=0.75, random_state=42), X, y, cv=10, scoring="accuracy"))

0.8349462365591398

# Grid Search CV

In [50]:
from sklearn.model_selection import GridSearchCV

In [52]:
params = {
    "n_estimators" : [20, 60, 100, 120],
    "max_features" : [0.2, 0.6, 1.0],
    "max_depth" : [2,8,None],
    "max_samples" : [0.5, 0.75, 1.0]
}

In [54]:
rf = RandomForestClassifier()

In [60]:
rf_grid = GridSearchCV(
    estimator=rf,
    param_grid=params,
    cv=5,
    verbose=2,
    n_jobs= -1
)

In [62]:
rf_grid

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 8, None],
                         'max_features': [0.2, 0.6, 1.0],
                         'max_samples': [0.5, 0.75, 1.0],
                         'n_estimators': [20, 60, 100, 120]},
             verbose=2)

In [64]:
rf_grid.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 8, None],
                         'max_features': [0.2, 0.6, 1.0],
                         'max_samples': [0.5, 0.75, 1.0],
                         'n_estimators': [20, 60, 100, 120]},
             verbose=2)

In [66]:
rf_grid.best_params_

{'max_depth': 2, 'max_features': 0.2, 'max_samples': 0.75, 'n_estimators': 100}

In [68]:
rf_grid.best_score_

0.8346938775510205

# Random Forest Learning

In [92]:
# Function for Row Sampling
def sample_rows(df, percent):
    return df.sample(int(percent*df.shape[0]), replace=True)

In [142]:
# Function for feature selection
def sample_features(df, percent):
    cols = random.sample(df.columns.tolist()[:-1], int(percent*df.shape[1]-1))
    new_df = df[cols]
    new_df["target"] = df["target"]
    return new_df

In [144]:
# Function for combine sampling
def sample_combine(df, row_percent, col_percent):
    new_df = sample_rows(df, row_percent)
    return sample_features(new_df, col_percent)

In [114]:
df.columns.tolist()

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'target']

In [86]:
0.75*303

227.25

In [90]:
int((303*75)/100)

227

In [136]:
df.columns.tolist()[:-1]

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

In [140]:
import random
random.sample(df.columns.tolist()[:-1], int(0.75*df.shape[1]-1))

['oldpeak', 'cp', 'thalach', 'thal', 'age', 'slope', 'chol', 'fbs', 'trestbps']

In [138]:
13*0.75

9.75